# imports

In [32]:
%pip install librosa
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
%reset -f

In [34]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import kagglehub

In [35]:
# Rutas
DATASET_PATH = "./data/"
LOGS_PATH = "./logs/fit/"
MODELS_PATH = "./models/"

In [36]:
# Función para extraer características
def extract_features(file_path):
    try:
        # Carga del archivo de audio
        audio, sr = librosa.load(file_path, mono=True, duration=30)
        
        # Extracción de características
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        chroma_mean = np.mean(chroma.T, axis=0)
        
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        spectral_mean = np.mean(spectral_contrast.T, axis=0)
        
        # Combinar todas las características
        features = np.hstack([mfccs_mean, chroma_mean, spectral_mean])
        return features
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

In [37]:
# Crear dataset
def prepare_dataset(dataset_path):
    data = []
    labels = []
    genres = os.listdir(dataset_path)
    
    for genre in genres:
        genre_path = os.path.join(dataset_path, genre)
        if not os.path.isdir(genre_path):
            continue
        
        for file in os.listdir(genre_path):
            file_path = os.path.join(genre_path, file)
            features = extract_features(file_path)
            if features is not None:
                data.append(features)
                labels.append(genre)
    
    return np.array(data), np.array(labels)

In [38]:
# Entrenamiento del modelo
def train_model(X, y):
    # División del dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenamiento de SVM
    svm = SVC(kernel='rbf', C=10, gamma=0.1,probability=True)
    svm.fit(X_train, y_train)
    
    # Evaluación
    y_pred = svm.predict(X_test)
    print("Reporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print(f"Precisión: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    
    return svm

In [39]:
# Visualización
def plot_features_distribution(features, labels, genres):
    plt.figure(figsize=(12, 6))
    for i, genre in enumerate(genres):
        plt.scatter(features[labels == genre, 0], features[labels == genre, 1], label=genre)
    plt.title("Distribución de características")
    plt.xlabel("Primera característica")
    plt.ylabel("Segunda característica")
    plt.legend()
    plt.show()

In [40]:
print("Preparando dataset...")
X, y = prepare_dataset(os.path.join(DATASET_PATH,"genres_original"))

# Verificar clases
unique_classes = np.unique(y)
print(f"Clases detectadas: {unique_classes}")

print("Entrenando modelo...")
model = train_model(X, y)

# (Opcional) Guardar el modelo entrenado
import joblib
joblib.dump(model, "genre_classifier_svm.pkl")
print("Modelo guardado como 'genre_classifier_svm.pkl'.")

Preparando dataset...


/tmp/ipykernel_27/3250408172.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, mono=True, duration=30)
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error procesando ./data/genres_original/jazz/jazz.00054.wav: 
Clases detectadas: ['blues' 'classical' 'country' 'disco' 'hiphop' 'jazz' 'metal' 'pop'
 'reggae' 'rock']
Entrenando modelo...
Reporte de clasificación:
              precision    recall  f1-score   support

       blues       0.00      0.00      0.00        21
   classical       0.06      1.00      0.12        12
     country       0.00      0.00      0.00        24
       disco       1.00      0.05      0.09        22
      hiphop       0.00      0.00      0.00        15
        jazz       0.00      0.00      0.00        27
       metal       0.67      0.11      0.19        18
         pop       1.00      0.05      0.10        19
      reggae       1.00      0.09      0.17        22
        rock       0.00      0.00      0.00        20

    accuracy                           0.09       200
   macro avg       0.37      0.13      0.07       200
weighted avg       0.38      0.09      0.06       200

Precisión: 9.00%
Modelo gu

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
file_path = "./data/genres_original/blues/"
results = []
for file in os.listdir(file_path):
    result = model.predict_proba(extract_features(file_path+file).reshape(1, -1))[0]
    clase = model.classes_[np.argmin(result)]
    prob = result[np.argmin(result)]
    results.append([clase,prob])
results


[['blues', 0.008885057507796001],
 ['blues', 0.008885219787507],
 ['blues', 0.008884457745351139],
 ['blues', 0.0088850216113248],
 ['blues', 0.008885251327690927],
 ['blues', 0.008884445075106544],
 ['blues', 0.008886393627319503],
 ['blues', 0.008885593801444724],
 ['blues', 0.008883746741306316],
 ['blues', 0.00888567556436225],
 ['pop', 0.09118993306597137],
 ['blues', 0.008883872973288886],
 ['blues', 0.00888512725005602],
 ['blues', 0.008884779357781843],
 ['blues', 0.008885451124826175],
 ['blues', 0.008885338980831813],
 ['blues', 0.008885608000541969],
 ['blues', 0.008885974216652437],
 ['blues', 0.008885007179421738],
 ['blues', 0.008882858054307973],
 ['blues', 0.008883889879637567],
 ['blues', 0.008884378861130509],
 ['blues', 0.008883871047483094],
 ['pop', 0.09118993306597137],
 ['blues', 0.008884983002205152],
 ['pop', 0.09118993306597137],
 ['blues', 0.008884269847324308],
 ['blues', 0.008885316895148638],
 ['blues', 0.008886449733380677],
 ['pop', 0.09118993306597137],